In [4]:
import mlflow
from mlflow import MlflowClient

client = MlflowClient()

# 1. Search for logged models ordered by class 1 recall
# Note: Use the experiment ID corresponding to "Wine_Quality_Locked_Dependencies"
# You can find the ID in the UI or by calling mlflow.get_experiment_by_name()
experiment = mlflow.get_experiment_by_name("Wine_Quality_Locked_Dependencies")
experiment_id = '1'

top_models = mlflow.search_logged_models(
    experiment_ids=[experiment_id],
    order_by=[{"field_name": "metrics.test_recall", "ascending": False}],
    max_results=1,
)

best_model_record = top_models.iloc[0]
model_name = "wine_quality_champion_model"

# 2. Register the top model
registered_version = mlflow.register_model(
    model_uri=f"runs:/{best_model_record.source_run_id}/model",
    name=model_name
)

# 3. Use MlflowClient to set description, tags, and alias
client.update_model_version(
    name=model_name,
    version=registered_version.version,
    description="Top performing model selected based on Class 1 Recall on test set."
)

client.set_model_version_tag(
    name=model_name,
    version=registered_version.version,
    key="role",
    value="champion"
)

client.set_registered_model_alias(
    name=model_name,
    alias="champion",
    version=registered_version.version
)

print(f"Model {model_name} version {registered_version.version} is now the Champion.")

Created version '2' of model 'wine_quality_champion_model'.


In [5]:

import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from mlflow import MlflowClient

# Initialize variables
client = MlflowClient()
model_name = "wine_quality_champion_model"

# --- 1. Re-load Data to define X_test and y_test ---
print("Loading dataset...")
wine = fetch_openml("wine-quality-red", version=1, as_frame=True)
X = wine.data
y = (wine.target.astype(int) >= 6).astype(int)

# Use the same random_state=42 to ensure the test set is identical to training
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- 2. Load the champion model using the alias ---
print(f"Loading champion model: {model_name}...")
champion_uri = f"models:/{model_name}@champion"
loaded_champion = mlflow.sklearn.load_model(champion_uri)

# --- 3. Make predictions on a small batch of test examples ---
sample_X = X_test.head(5)
predictions = loaded_champion.predict(sample_X)

print("\n--- Predictions on Sample Data ---")
results_df = sample_X.copy()
results_df["predicted_quality"] = predictions
results_df["actual_quality"] = y_test.head(5).values
print(results_df[["predicted_quality", "actual_quality"]])

# --- 4. Retrieve and print metadata to verify ---
print("\n--- Champion Metadata Verification ---")
try:
    model_metadata = client.get_model_version_by_alias(model_name, "champion")
    print(f"Alias 'champion' points to Version: {model_metadata.version}")
    print(f"Source Run ID: {model_metadata.run_id}")
    print(f"Tags: {model_metadata.tags}")

    # Verify the source (e.g., if it came from experiment 1)
    run = mlflow.get_run(model_metadata.run_id)
    print(f"Original Experiment ID: {run.info.experiment_id}")

except Exception as e:
    print(f"Error retrieving metadata: {e}")

Loading dataset...
Loading champion model: wine_quality_champion_model...

--- Predictions on Sample Data ---
      predicted_quality  actual_quality
803                   0               1
124                   0               0
350                   0               1
682                   0               0
1326                  1               1

--- Champion Metadata Verification ---
Alias 'champion' points to Version: 2
Source Run ID: 1002e0f1d57e4a438e08e62e42f7c3c0
Tags: {'role': 'champion'}
Original Experiment ID: 1
